# <b>Road Damage Detection Using YOLOv5</b>


---

The objective is to provide a robust Single Shot Detector which can detect road damages across multiple country's road condition 

In this notebook, all the cells can be run at one shot. It will stop you whereever required.

Ref: https://towardsai.net/p/computer-vision/yolo-v5-object-detection-on-a-custom-dataset

Ref: https://colab.research.google.com/github/bala-codes/Yolo-v5_Object_Detection_Blood_Cell_Count_and_Detection/blob/master/codes/1.%20Yolo-V5%20BCC%20Training%20%26%20Testing.ipynb#scrollTo=NSAegF-48fHj

Ref: https://colab.research.google.com/drive/16QCaYzTuHCOF9CQLQYmGNxmtY1xKAIdn?usp=sharing#scrollTo=7yQ2AiKJfhLg

In [ ]:
from IPython.display import clear_output 
import pandas as pd
import os
from glob import glob
from sklearn import preprocessing, model_selection
import numpy as np
import shutil
from PIL import Image


###Data Load & Preprocessing


In [ ]:
!unzip '/content/drive/MyDrive/colabnotebooks/road_damage_detection/aug_images.zip' -d '/'
augmented_images_df = pd.read_csv('/content/augmented_images_df.csv')
clear_output()

In [ ]:
augmented_images_df = pd.read_csv('/content/augmented_images_df.csv')
augmented_images_df=augmented_images_df[['filename',	'width',	'height',	'class',	'xmin',	'ymin',	'xmax',	'ymax']]
augmented_images_df.dropna(axis=0, inplace=True)

In [ ]:
## remove unnecessary images
#tx=glob('/content/aug_images/*.txt')
img=glob('/content/aug_images/*.jpg')

img=set([i.replace('/content/aug_images/','').replace('.jpg','') for i in img])
#tx=set([i.replace('/content/aug_images/','').replace('.txt','') for i in tx])

tbd=img-set(augmented_images_df.filename.apply(lambda x : x.replace('.jpg', '')).values)

tbd = ['/content/aug_images/'+i+'.jpg' for i in tbd]
for i in tbd:
  print(i+' Deleted')
  os.remove(i)


for i in glob('/content/aug_images/*.txt'):
  os.remove(i)

clear_output()

In [ ]:
#test

img=glob('/content/aug_images/*.jpg')

img=set([i.replace('/content/aug_images/','').replace('.jpg','') for i in img])
img-set(augmented_images_df.filename.apply(lambda x : x.replace('.jpg', '')).values)


set()

In [ ]:
img_width = 416
img_height = 416

def width(df):
  return int(df.xmax - df.xmin)
def height(df):
  return int(df.ymax - df.ymin)
def x_center(df):
  return int(df.xmin + (df.width/2))
def y_center(df):
  return int(df.ymin + (df.height/2))
def w_norm(df):
  return df/img_width
def h_norm(df):
  return df/img_height

df = augmented_images_df

le = preprocessing.LabelEncoder()
le.fit(df['class'])
print(le.classes_)
labels = le.transform(df['class'])
df['labels'] = labels

df['width'] = df.apply(width, axis=1)
df['height'] = df.apply(height, axis=1)

df['x_center'] = df.apply(x_center, axis=1)
df['y_center'] = df.apply(y_center, axis=1)

df['x_center_norm'] = df['x_center'].apply(w_norm)
df['width_norm'] = df['width'].apply(w_norm)

df['y_center_norm'] = df['y_center'].apply(h_norm)
df['height_norm'] = df['height'].apply(h_norm)

df.head()


['D00' 'D01' 'D10' 'D11' 'D20' 'D40' 'D43' 'D44']


,filename,width,height,class,xmin,ymin,xmax,ymax,labels,x_center,y_center,x_center_norm,width_norm,y_center_norm,height_norm
0,aug1_Japan_000484.jpg,46,93,D00,136.68001,0.000000,183.133330,93.33333,0,159,46,0.382212,0.110577,0.110577,0.223558
1,aug1_Japan_002789.jpg,20,38,D00,277.72000,231.573330,297.826660,270.40000,0,287,250,0.689904,0.048077,0.600962,0.091346
2,aug1_Japan_002762.jpg,91,146,D00,324.26040,195.387470,416.000000,341.40384,0,369,268,0.887019,0.218750,0.644231,0.350962
3,aug1_Japan_009702.jpg,55,54,D00,0.00000,352.213350,55.160004,406.29333,0,27,379,0.064904,0.132212,0.911058,0.129808
4,aug1_Japan_000613.jpg,31,128,D00,0.00000,97.520004,31.306671,226.47998,0,15,161,0.036058,0.074519,0.387019,0.307692


In [ ]:
df_train, df_valid = model_selection.train_test_split(df, test_size=0.1, random_state=13, shuffle=True,
                                                      stratify=df.labels)
print(df_train.shape, df_valid.shape)

os.mkdir('/content/road_damage_detection/')
os.mkdir('/content/road_damage_detection/images/')
os.mkdir('/content/road_damage_detection/images/train/')
os.mkdir('/content/road_damage_detection/images/valid/')

os.mkdir('/content/road_damage_detection/labels/')
os.mkdir('/content/road_damage_detection/labels/train/')
os.mkdir('/content/road_damage_detection/labels/valid/')

def segregate_data(df, img_path, label_path, train_img_path, train_label_path):
  filenames = []
  for filename in df.filename:
    filenames.append(filename)
  filenames = set(filenames)
  
  for filename in filenames:
    yolo_list = []

    for _,row in df[df.filename == filename].iterrows():
      yolo_list.append([row.labels, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])

    yolo_list = np.array(yolo_list)
    txt_filename = os.path.join(train_label_path,str(row.filename.split('.')[0])+".txt")
    # Save the .img & .txt files to the corresponding train and validation folders
    np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])
    shutil.copyfile(os.path.join(img_path,row.filename), os.path.join(train_img_path,row.filename))
 
## Apply function ## 
src_img_path = "/content/aug_images/"
src_label_path = "/content/aug_images/"

train_img_path = "/content/road_damage_detection/images/train"
train_label_path = "/content/road_damage_detection/labels/train"

valid_img_path = "/content/road_damage_detection/images/valid"
valid_label_path = "/content/road_damage_detection/labels/valid"

segregate_data(df_train, src_img_path, src_label_path, train_img_path, train_label_path)
segregate_data(df_valid, src_img_path, src_label_path, valid_img_path, valid_label_path)

print("No. of Training images", len(os.listdir('/content/road_damage_detection/images/train')))
print("No. of Training labels", len(os.listdir('/content/road_damage_detection/labels/train')))

print("No. of valid images", len(os.listdir('/content/road_damage_detection/images/valid')))
print("No. of valid labels", len(os.listdir('/content/road_damage_detection/labels/valid')))


(13874, 15) (1542, 15)
No. of Training images 10501
No. of Training labels 10501
No. of valid images 1493
No. of valid labels 1493


In [ ]:
!cp -r /content/road_damage_detection/labels/valid/ /content/drive/MyDrive/colabnotebooks/road_damage_detection/valid

In [ ]:
len(glob('/content/drive/MyDrive/colabnotebooks/road_damage_detection/valid/*.txt'))


1493

In [ ]:
!rm -rf /content/aug_images

### Model — Training


In [ ]:
%cd /content

!git clone  'https://github.com/ultralytics/yolov5.git'

!pip install -qr '/content/yolov5/requirements.txt'  # install dependencies

%cd /content/yolov5

## Create a yaml file and move it into the yolov5 folder ##
!echo -e 'train: /content/road_damage_detection/images/train\nval: /content/road_damage_detection/images/valid\n\nnc: 8\nnames: ['D00', 'D01', 'D10', 'D11', 'D20', 'D40', 'D43', 'D44']' >> rdd.yaml
!cat 'rdd.yaml'
# shutil.copyfile('/content/rdd.yaml', '/content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/Installed_Model/yolov5/rdd.yaml')


# Also edit the number of classes (nc) in the ./models/*.yaml file
# Choose the yolo model of your choice, here I chose yolov5s.yaml (yolo - small)
!sed -i 's/nc: 80/nc: 8/g' /content/yolov5/models/yolov5s.yaml


/content
Cloning into 'yolov5'...
remote: Enumerating objects: 6220, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 6220 (delta 4), reused 2 (delta 1), pack-reused 6206
Receiving objects: 100% (6220/6220), 8.48 MiB | 10.91 MiB/s, done.
Resolving deltas: 100% (4259/4259), done.
     |████████████████████████████████| 645kB 11.3MB/s 
/content/yolov5
train: /content/road_damage_detection/images/train
val: /content/road_damage_detection/images/valid

nc: 8
names: [D00, D01, D10, D11, D20, D40, D43, D44]


In [ ]:
d=max(os.listdir('/content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights'))
cmd='train.py --img 416 --batch 8 --epochs 100 --data rdd.yaml --cfg models/yolov5s.yaml --name RDD_yolov5 --project /content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights --weights /content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights/{}/weights/last.pt'.format(d)
cmd

'train.py --img 416 --batch 8 --epochs 100 --data rdd.yaml --cfg models/yolov5s.yaml --name RDD_yolov5 --project /content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights --weights /content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights/RDD_yolov55/weights/last.pt'

In [ ]:
%cd /content/yolov5
!python $cmd

/content/yolov5
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-31-g1b1ab4c torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=False, cfg='models/yolov5s.yaml', data='rdd.yaml', device='', entity=None, epochs=100, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='RDD_yolov5', noautoanchor=False, nosave=False, notest=False, project='/content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights', quad=False, rect=False, resume=False, save_dir='/content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights/RDD_yolov56', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=8, upload_dataset=False, weights='/content/drive/M

In [ ]:
# Start tensorboard (optional)
# %load_ext tensorboard
# %tensorboard --logdir runs

# %cd /content/yolov5/
# !python train.py --img 416 --batch 8 --epochs 100 --data bcc.yaml --cfg models/yolov5s.yaml --name RDD_yolov5 --project /content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights --weights /content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights/RDD_yolov5/weights/best.pt

Interference with a Video

In [ ]:
%cd /content/
#!wget https://vod-progressive.akamaized.net/exp=1620149194~acl=%2Fvimeo-prod-skyfire-std-us%2F01%2F1013%2F18%2F455069400%2F2006278284.mp4~hmac=dbf320a4d936fdb04c2e638f71a69f5e9cd91631083d5c6f0c0a66902c13004e/vimeo-prod-skyfire-std-us/01/1013/18/455069400/2006278284.mp4?download=1&filename=pexels-kelly-lacy-5283466.mp4

#!wget https://vod-progressive.akamaized.net/exp=1620138268~acl=%2Fvimeo-prod-skyfire-std-us%2F01%2F3610%2F13%2F343052045%2F1373111307.mp4~hmac=8f199c30722ec650e4354e65a18d25c12a1d93ab694bdece74aa954c79adba62/vimeo-prod-skyfire-std-us/01/3610/13/343052045/1373111307.mp4?download=1&filename=Pexels+Videos+2519660.mp4

!wget https://content.videvo.net/videvo_files/video/free/2017-10/originalContent/170609_E_Varanasi_090_1.mp4

/content
--2021-05-09 14:23:11--  https://content.videvo.net/videvo_files/video/free/2017-10/originalContent/170609_E_Varanasi_090_1.mp4
Resolving content.videvo.net (content.videvo.net)... 104.18.26.42, 104.18.27.42, 2606:4700::6812:1a2a, ...
Connecting to content.videvo.net (content.videvo.net)|104.18.26.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78464855 (75M) [video/mp4]
Saving to: ‘170609_E_Varanasi_090_1.mp4’

170609_E_Varanasi_0 100%[===================>]  74.83M  30.0MB/s    in 2.5s    

2021-05-09 14:23:13 (30.0 MB/s) - ‘170609_E_Varanasi_090_1.mp4’ saved [78464855/78464855]



In [ ]:
%cd /content/yolov5
!python detect.py --source /content/170609_E_Varanasi_090_1.mp4 --weights /content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights/RDD_yolov55/weights/last.pt

/content/yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/170609_E_Varanasi_090_1.mp4', update=False, view_img=False, weights=['/content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights/RDD_yolov55/weights/last.pt'])
YOLOv5 🚀 v5.0-76-g57b0d3a torch 1.8.1+cu101 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7072789 parameters, 0 gradients
video 1/1 (1/848) /content/170609_E_Varanasi_090_1.mp4: 384x640 Done. (0.048s)
video 1/1 (2/848) /content/170609_E_Varanasi_090_1.mp4: 384x640 Done. (0.028s)
video 1/1 (3/848) /content/170609_E_Varanasi_090_1.mp4: 384x640 Done. (0.028s)
video 1/1 (4/848) /content/170609_E_Varanasi_090_1.mp4: 384x640 Done. (0.028s)
vide

In [ ]:
!cp /content/yolov5/runs/detect/exp2/1373111307.mp4 /content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5

another way of interference

In [ ]:
import torch

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/trained_weights/RDD_yolov55/weights/last.pt')  # custom model


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 224 layers, 7072789 parameters, 0 gradients
Adding autoShape... 
YOLOv5 🚀 2021-5-6 torch 1.8.1+cu101 CUDA:0 (Tesla K80, 11441.1875MB)



In [ ]:
model.eval()

def get_prediction(img_bytes):
    img = Image.open(img_bytes)
    imgs = [img]  # batched list of images

# Inference
    results = model(imgs, size=640)  # includes NMS
    return results


In [ ]:
import io

img_bytes = '/content/aug_images/aug10_India_000038.jpg'
results = get_prediction(img_bytes)
results.save()  # save as results1.jpg, results2.jpg... etc.


In [ ]:
model('/content/drive/MyDrive/colabnotebooks/road_damage_detection/Road_Damage_Detection_YOLOv5/1373111307.mp4')